# Améliorez le produit IA de votre start-up

## Analyser les photos pour déterminer les catégories des photos 

## Extaire le jeu de données utile

### Importer la librairie

In [1]:
import numpy as np
import pandas as pd
import json
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import os
from os import listdir
from matplotlib.image import imread
from PIL import Image
from PIL import ImageFilter
import cv2
from sklearn.cluster import KMeans
from scipy.spatial import distance
from sklearn import cluster, metrics
from sklearn.metrics import silhouette_score
from sklearn import manifold, decomposition

In [2]:
datafile = '../P6/photos.json'
img_dir = '../P6/photos/'
outputfile = '../P6/P6_01_fichiercsv_photos_01.csv'
outputfile_raw = '../P6/P6_01_fichiercsv_photos_02.csv'
chunksize = 10000

# Chargement des données
Dans un premier temps on charge les données qui sont dans un fichier photos.json et l'on ne prend que l'indentifiant de l'image (photo_id) et le label de cette image (label). C'est données sont chargée dans un data frame.

In [3]:
keep_cols = ['photo_id','label']
header = True
i = 0
for chunk in pd.read_json(datafile, chunksize=chunksize,orient='records', lines=True):
        chunk.to_json(str(i) + '.json')
        chunk = chunk[keep_cols]
        rawcsv = chunk[keep_cols]
        if header:
            chunk.to_csv(outputfile, index=False)        
            rawcsv.to_csv(outputfile_raw, index=False)        
        else:
            chunk.to_csv(outputfile, mode='a', header=False, index=False)
            rawcsv.to_csv(outputfile_raw, mode='a', header=False, index=False)
        i += 1
        header = False    

In [4]:
data = pd.read_csv('../P6/P6_01_fichiercsv_photos_02.csv')
data.head()

,photo_id,label
0,Un_Og6jfhazVn7CxszkKEw,drink
1,BFE1AFOs27scnnfeBf99ZA,drink
2,7t-C0r1JRdoVD9FS7M-N7Q,drink
3,rLnw0d-YYZvT9kR4y7h7_Q,drink
4,Cv5M8MDw8a5NEWvw2AQ4nw,drink


# Exploration des données
Le jeu de données contient 200 000 images, une seule image est dupliquée. Il y a 5 catégories :
 - interior
 - food
 - outside
 - menu
 - drink

Les images sont équitablement réparties avec dans les catégories (40 000 de chaque)

In [5]:
print(f"{data['photo_id'].shape[0]} images")
print(f"{data.duplicated(subset=['photo_id']).sum()} images dupliquées")

print("")
print("Données manquantes")
print(f"{data.isna().sum()}")
print("")
print(f"{len(data['label'].unique())} labels")
print(data['label'].unique())

200000 images
1 images dupliquées

Données manquantes
photo_id    0
label       0
dtype: int64

5 labels
['drink' 'food' 'interior' 'outside' 'menu']


In [6]:
data['label'].value_counts()

drink       40000
food        40000
interior    40000
outside     40000
menu        40000
Name: label, dtype: int64

# Ajout d'une colonne pour les categories
Nous ajoutons une colonne label_num qui corespond aux labels sous forme numérique. Cette colonne servira lors de l'&valuation des prédictions

In [7]:
data['label_num'] = data['label'].map({
    'drink'   :0,
    'menu'    :1,
    'food'    :2,
    'outside' :3,
    'interior':4
})

## Extrait des données d'entrainement
Pour cette exploration nous utiliserons un jeu de donnés d'entrainement partiel de 500 images par categories, soit 2500 images au total

In [8]:
# Extrait des données
n_sample = 500
df_drink = data[data['label'] == 'drink'].sample(n=n_sample)
df_menu = data[data['label'] == 'menu'].sample(n=n_sample)
df_interior = data[data['label'] == 'interior'].sample(n=n_sample)
df_outside = data[data['label'] == 'outside'].sample(n=n_sample)
df_food = data[data['label'] == 'food'].sample(n=n_sample)

data_train = pd.concat([df_drink, df_menu, df_interior,df_outside, df_food ], ignore_index=True)

In [9]:
data_train['label'].value_counts()

drink       500
menu        500
interior    500
outside     500
food        500
Name: label, dtype: int64

## Création des labels des photos à partir de photo_id d'entrainement

In [10]:
for index, row in data_train.iterrows():
    file = ( data_train.at[index,'photo_id'] + ".jpg")
    row['fichiers_photo'] = file
    data_train.at[index,'fichiers_photo'] = row['fichiers_photo']  

In [11]:
data_train

,photo_id,label,label_num,fichiers_photo
0,uq1qcV5kf36rjB1TP1GCLg,drink,0,uq1qcV5kf36rjB1TP1GCLg.jpg
1,ZW5jolpNvcUp-9WTDvflzA,drink,0,ZW5jolpNvcUp-9WTDvflzA.jpg
2,IGXvDVk-jO-Gf2kJSK430Q,drink,0,IGXvDVk-jO-Gf2kJSK430Q.jpg
3,4jYV6b64iMhduZHP-gt--Q,drink,0,4jYV6b64iMhduZHP-gt--Q.jpg
4,708W7D3giVuge7Qh38BHVQ,drink,0,708W7D3giVuge7Qh38BHVQ.jpg
...,...,...,...,...
2495,mz57yXcVE4g32W7UuaUIQA,food,2,mz57yXcVE4g32W7UuaUIQA.jpg
2496,TbyfSlEc45NGH-c2LxBgsg,food,2,TbyfSlEc45NGH-c2LxBgsg.jpg
2497,3mCg7Bh0u37WHNZclEFeyg,food,2,3mCg7Bh0u37WHNZclEFeyg.jpg
2498,w8p1cpRO0b167R4mauXiQQ,food,2,w8p1cpRO0b167R4mauXiQQ.jpg


## Supression de photo id dans les données d'entrainement et sauvegarde en fichier CSV

In [12]:
data_train = data_train.drop(["photo_id"], axis=1)

data_train.to_csv (r'P6_01_fichiercsv_photos_03_train.csv', index = False, header=True)

In [13]:
data_train

,label,label_num,fichiers_photo
0,drink,0,uq1qcV5kf36rjB1TP1GCLg.jpg
1,drink,0,ZW5jolpNvcUp-9WTDvflzA.jpg
2,drink,0,IGXvDVk-jO-Gf2kJSK430Q.jpg
3,drink,0,4jYV6b64iMhduZHP-gt--Q.jpg
4,drink,0,708W7D3giVuge7Qh38BHVQ.jpg
...,...,...,...
2495,food,2,mz57yXcVE4g32W7UuaUIQA.jpg
2496,food,2,TbyfSlEc45NGH-c2LxBgsg.jpg
2497,food,2,3mCg7Bh0u37WHNZclEFeyg.jpg
2498,food,2,w8p1cpRO0b167R4mauXiQQ.jpg
